In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('SparkDataFrameDemo').getOrCreate()

22/05/15 21:19:56 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.108.128 instead (on interface ens33)
22/05/15 21:19:56 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/15 21:19:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
input_path = "Admission_Predict.csv"
df = spark.read.option('header', True).csv(input_path)
df.take(5)

[Row(Serial No.='1', GRE Score='337', TOEFL Score='118', University Rating='4', SOP='4.5', LOR ='4.5', CGPA='9.65', Research='1', Chance of Admit ='0.92'),
 Row(Serial No.='2', GRE Score='324', TOEFL Score='107', University Rating='4', SOP='4', LOR ='4.5', CGPA='8.87', Research='1', Chance of Admit ='0.76'),
 Row(Serial No.='3', GRE Score='316', TOEFL Score='104', University Rating='3', SOP='3', LOR ='3.5', CGPA='8', Research='1', Chance of Admit ='0.72'),
 Row(Serial No.='4', GRE Score='322', TOEFL Score='110', University Rating='3', SOP='3.5', LOR ='2.5', CGPA='8.67', Research='1', Chance of Admit ='0.8'),
 Row(Serial No.='5', GRE Score='314', TOEFL Score='103', University Rating='2', SOP='2', LOR ='3', CGPA='8.21', Research='0', Chance of Admit ='0.65')]

In [7]:
df.printSchema()

root
 |-- Serial No.: string (nullable = true)
 |-- GRE Score: string (nullable = true)
 |-- TOEFL Score: string (nullable = true)
 |-- University Rating: string (nullable = true)
 |-- SOP: string (nullable = true)
 |-- LOR : string (nullable = true)
 |-- CGPA: string (nullable = true)
 |-- Research: string (nullable = true)
 |-- Chance of Admit : string (nullable = true)



In [8]:
df.show()

+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|         1|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|
|         2|      324|        107|                4|  4| 4.5|8.87|       1|            0.76|
|         3|      316|        104|                3|  3| 3.5|   8|       1|            0.72|
|         4|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|
|         5|      314|        103|                2|  2|   3|8.21|       0|            0.65|
|         6|      330|        115|                5|4.5|   3|9.34|       1|             0.9|
|         7|      321|        109|                3|  3|   4| 8.2|       1|            0.75|
|         8|      308|        101|                2|  3|   4| 7.9|    

In [9]:
df.createOrReplaceTempView("Graduate")

In [10]:
df2 = spark.sql("SELECT * FROM Graduate WHERE Research=1")
df2.show()

+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit |
+----------+---------+-----------+-----------------+---+----+----+--------+----------------+
|         1|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|
|         2|      324|        107|                4|  4| 4.5|8.87|       1|            0.76|
|         3|      316|        104|                3|  3| 3.5|   8|       1|            0.72|
|         4|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|
|         6|      330|        115|                5|4.5|   3|9.34|       1|             0.9|
|         7|      321|        109|                3|  3|   4| 8.2|       1|            0.75|
|        11|      325|        106|                3|3.5|   4| 8.4|       1|            0.52|
|        12|      327|        111|                4|  4| 4.5|   9|    

In [11]:
# we can drop the temprory view
spark.catalog.dropTempView("Graduate")

In [12]:
from pyspark.sql.functions import exp, lit 
df_with_nc1 = df.withColumn("n1", lit(0))
df_with_nc2 = df_with_nc1.withColumn("n2", exp("CGPA"))

In [13]:
df_with_nc1.show()

+----------+---------+-----------+-----------------+---+----+----+--------+----------------+---+
|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit | n1|
+----------+---------+-----------+-----------------+---+----+----+--------+----------------+---+
|         1|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|  0|
|         2|      324|        107|                4|  4| 4.5|8.87|       1|            0.76|  0|
|         3|      316|        104|                3|  3| 3.5|   8|       1|            0.72|  0|
|         4|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|  0|
|         5|      314|        103|                2|  2|   3|8.21|       0|            0.65|  0|
|         6|      330|        115|                5|4.5|   3|9.34|       1|             0.9|  0|
|         7|      321|        109|                3|  3|   4| 8.2|       1|            0.75|  0|
|         8|      308|        

In [14]:
df_with_nc2.show()

+----------+---------+-----------+-----------------+---+----+----+--------+----------------+---+------------------+
|Serial No.|GRE Score|TOEFL Score|University Rating|SOP|LOR |CGPA|Research|Chance of Admit | n1|                n2|
+----------+---------+-----------+-----------------+---+----+----+--------+----------------+---+------------------+
|         1|      337|        118|                4|4.5| 4.5|9.65|       1|            0.92|  0|15521.788104196934|
|         2|      324|        107|                4|  4| 4.5|8.87|       1|            0.76|  0| 7115.280973169776|
|         3|      316|        104|                3|  3| 3.5|   8|       1|            0.72|  0|2980.9579870417283|
|         4|      322|        110|                3|3.5| 2.5|8.67|       1|             0.8|  0| 5825.499349524731|
|         5|      314|        103|                2|  2|   3|8.21|       0|            0.65|  0|3677.5424662662012|
|         6|      330|        115|                5|4.5|   3|9.34|      

In [15]:
#spark.stop()